In [166]:
import import_ipynb
from ConsistencyIndexes import *

In [167]:
class ParamInvividual:
    def __init__(self,N=2,idx=[CRC,Border_Err],W=[1,1],M="mean"):
        self.NIdx = N
        self.Idx = idx
        self.Weights = W
        self.Mode = M
        self.Score = 0

In [168]:
def CopyIndv(Inv):
    return ParamInvividual(Inv.NIdx,Inv.Idx,Inv.Weights,Inv.Mode)

In [169]:
def Generator():
    Indexes = [TSNR,Abs_Dif,OF_EPE,OF_AE,Gray_Dif,TSSIM,MSE,Border_Err,CRC,Entopy_Dif]
    Modes = ["mean","log","norm","Z"]
    Nindex = random.sample(range(0,len(Indexes)),random.randint(2,len(Indexes)))
    Idx = [Indexes[i] for i in range(len(Nindex))]
    W = [random.uniform(0.1, 10) for _ in range(len(Nindex))]
    M = Modes[random.randint(0,len(Modes)-1)]
    return ParamInvividual(len(Nindex),Idx,W,M)

In [170]:
def Gen_Population(N=100):
    return [Generator() for _ in range(N)]

In [171]:
def Evaluation(Individual,Frames,Targets):
    Inc,Metric = InconsistentRegion(Frames,Mix_Metrics,Individual.Mode,Individual.Idx,Individual.Weights)
    Inc = Inc[1:]
    Dif = [abs(Inc[i]-Targets[i]) for i in range(len(Inc))]
    return np.mean(sum(Dif)/len(Dif))# + Metric)

In [172]:
def Evaluate_Population(Population,Frames,Targets):
    for i in range(len(Population)):
        print("Pop: ",i+1,'/',len(Population),'\n')
        Population[i].Score = Evaluation(Population[i],Frames,Targets)

In [173]:
def Read2Binary(path):
    T = read_images(path)
    T = [cv2.cvtColor(t,cv2.COLOR_RGB2GRAY) for t in T]
    for t in T:
        t[t>=150]=255
        t[t<150]=0
    return T

In [174]:
def Selection(Population,NTop=10):
    Scores = [P.Score for P in Popu]
    Dictionary = dict(zip(Popu, Scores))
    Sorted = sorted(Dictionary.items(), key=lambda x: x[1])
    return [key for key, value in Sorted[:NTop]]

In [175]:
def Cross(Indv1,Indv2):
    N = (Indv1.NIdx+Indv2.NIdx)//2
    Dic = dict(zip(Indv1.Idx+Indv2.Idx,Indv1.Weights+Indv2.Weights))
    idx = random.sample(list(Dic.keys()), N)
    W = [Dic[key] for key in idx]
    M = [Indv1.Mode,Indv2.Mode][random.randint(0,1)]
    return ParamInvividual(N,idx,W,M)

In [176]:
def Mutation(Indv):
    I = CopyIndv(Indv)
    I.Weights = [random.uniform(0.1, 10) for _ in range(Indv.NIdx)]
    return I

In [177]:
def Gen_From_Top(N,Top):
    Population = []
    for n in range(N):
        Op = random.randint(0,1)
        if Op==0:
            I = Cross(Top[random.randint(0,len(Top)-1)],Top[random.randint(0,len(Top)-1)])
        else:
            I = Mutation(Top[random.randint(0,len(Top)-1)])
        Population.append(I)
    return Population

In [178]:
def Train(Frames,Targets,NPop=5,Generations=3,NTop=3):
    Population = Gen_Population(NPop)
    Evaluate_Population(Population,Frames,Targets)
    for G in range(Generations):
        print("Gen: ",G+1,"/",Generations,'\n')
        Top = Selection(Population,NTop)
        Children = Gen_From_Top(NPop-NTop,Top)
        Evaluate_Population(Children,Frames,Targets)
        Population = Top+Children
    Top = Selection(Population,NTop)
    return Top[0]

In [179]:
Frames = read_images("saved_frames")
Frames[3] = Frames[3][:,:,:3]
Frames[6] = Frames[6][:,:,:3]
Frames[9] = Frames[9][:,:,:3]
M = Read2Binary("Masks")
Masks = [np.zeros((M[0].shape[0],M[0].shape[1])) for _ in range(len(Frames)-1)]
Masks[2] = M[0]
Masks[5] = M[1]
Masks[8] = M[2]

In [180]:
Best = Train(Frames[:10],Masks[:9],20,10,5)

Pop:  1 / 20 

Pop:  2 / 20 

Pop:  3 / 20 

Pop:  4 / 20 

Pop:  5 / 20 

Pop:  6 / 20 

Pop:  7 / 20 

Pop:  8 / 20 

Pop:  9 / 20 

Pop:  10 / 20 

Pop:  11 / 20 

Pop:  12 / 20 

Pop:  13 / 20 

Pop:  14 / 20 

Pop:  15 / 20 

Pop:  16 / 20 

Pop:  17 / 20 

Pop:  18 / 20 

Pop:  19 / 20 

Pop:  20 / 20 

Gen:  1 / 10 

Pop:  1 / 15 

Pop:  2 / 15 

Pop:  3 / 15 

Pop:  4 / 15 

Pop:  5 / 15 

Pop:  6 / 15 

Pop:  7 / 15 

Pop:  8 / 15 

Pop:  9 / 15 

Pop:  10 / 15 

Pop:  11 / 15 

Pop:  12 / 15 

Pop:  13 / 15 

Pop:  14 / 15 

Pop:  15 / 15 

Gen:  2 / 10 

Pop:  1 / 15 

Pop:  2 / 15 

Pop:  3 / 15 

Pop:  4 / 15 

Pop:  5 / 15 

Pop:  6 / 15 

Pop:  7 / 15 

Pop:  8 / 15 

Pop:  9 / 15 

Pop:  10 / 15 

Pop:  11 / 15 

Pop:  12 / 15 

Pop:  13 / 15 

Pop:  14 / 15 

Pop:  15 / 15 

Gen:  3 / 10 

Pop:  1 / 15 

Pop:  2 / 15 

Pop:  3 / 15 

Pop:  4 / 15 

Pop:  5 / 15 

Pop:  6 / 15 

Pop:  7 / 15 

Pop:  8 / 15 

Pop:  9 / 15 

Pop:  10 / 15 

Pop:  11 / 15 

Pop:  12 / 15 


KeyboardInterrupt: 

In [ ]:
 Img = DrawInconsistancy1(Frames[:13],Mix_Metrics,Best.Mode,Best.Idx,Best.Weights)

In [ ]:
ThroughFrames(Img)